## Data Cleaning Part3

### Author: Lucia Zou
### Contact: lucia.zouyuebca@gmail.com 
### Date: Nov.1st, 2023

### Table of Contents
- [Volume Columns Cleaning](#Volume-Columns-Cleaning)
- [Item Columns Cleaning](#Item-Columns-Cleaning)
- [State Columns Cleaning](#State-Columns-Cleaning)

In [2]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [74]:
df6=pd.read_parquet('IowaAfterClean2.parquet')

In [6]:
df6.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,county_number,county,category,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-33179700135,2021-01-04,2576,Hy-Vee Wine and Spirits / Storm Lake,1250 N Lake St,Storm Lake,50588.0,11.0,Adair,1081600.0,...,64870,Fireball Cinnamon Whiskey,48,100,0.90,1.35,48,64.80,4.8,1.26
1,INV-33196200106,2021-01-04,2649,Hy-Vee #3 / Dubuque,400 Locust St,Dubuque,52001.0,31.0,Adair,1081200.0,...,65200,Tequila Rose Liqueur,12,750,11.50,17.25,4,69.00,3.0,0.79
2,INV-33184300011,2021-01-04,2539,Hy-Vee Food Store / Iowa Falls,640 S. Oak,Iowa Falls,50126.0,42.0,Adams,1031100.0,...,38008,Smirnoff 80prf PET,6,1750,14.75,22.13,6,132.78,10.5,2.77
3,INV-33184100015,2021-01-04,4024,Wal-Mart 1546 / Iowa Falls,840 S Oak,Iowa Falls,50126.0,42.0,Adams,1031100.0,...,36648,Caliber Vodka,12,750,3.31,4.97,12,59.64,9.0,2.37
4,INV-33174200025,2021-01-04,5385,Vine Food & Liquor,2704 Vine St.,West Des Moines,50265.0,77.0,Allamakee,1012200.0,...,4626,Buchanan Deluxe 12YR,12,750,20.99,31.49,2,62.98,1.5,0.39


In [17]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2805303 entries, 0 to 2805306
Data columns (total 23 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   invoice_and_item_number  object 
 1   date                     object 
 2   store_number             int64  
 3   store_name               object 
 4   address                  object 
 5   city                     object 
 6   zip_code                 float64
 7   county_number            float64
 8   county                   object 
 9   category                 float64
 10  category_name            object 
 11  vendor_number            float64
 12  vendor_name              object 
 13  item_number              int64  
 14  item_description         object 
 15  pack                     int64  
 16  bottle_volume_ml         int64  
 17  state_bottle_cost        float64
 18  state_bottle_retail      float64
 19  bottles_sold             int64  
 20  sale_dollars             float64
 21  volume_s

In [15]:
df6.nunique()

invoice_and_item_number    2805303
date                           326
store_number                  1929
store_name                    1952
address                       1936
city                           442
zip_code                       479
county_number                   99
county                          99
category                        47
category_name                   47
vendor_number                  222
vendor_name                    222
item_number                   4543
item_description              4029
pack                            21
bottle_volume_ml                25
state_bottle_cost             1312
state_bottle_retail           1315
bottles_sold                   397
sale_dollars                 11136
volume_sold_liters             772
volume_sold_gallons            761
dtype: int64

###

### Volume Columns Cleaning

In [75]:
crosst= pd.crosstab(df6['volume_sold_liters'], df6['volume_sold_gallons'])

In [76]:
columns_with_multiple_nonzero = crosst.columns[crosst.gt(0).sum(axis=0) > 1]
columns_with_multiple_nonzero

Float64Index([0.01, 0.02, 0.03, 0.06, 0.09, 0.19, 0.29, 0.89, 1.08, 2.27], dtype='float64', name='volume_sold_gallons')

In [77]:
gallons_values = [0.01, 0.02, 0.03, 0.06, 0.09, 0.19, 0.29, 0.89, 1.08, 2.27]

df6[df6['volume_sold_gallons'] == 0.01]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 0.02]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 0.03]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 0.06]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 0.09]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 0.19]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 0.29]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 0.89]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 1.08]['volume_sold_liters'].unique()
df6[df6['volume_sold_gallons'] == 2.27]['volume_sold_liters'].unique()

array([0.05, 0.06, 0.04])

array([0.1 , 0.08])

array([0.15, 0.12])

array([0.24, 0.25])

array([0.37, 0.35])

array([0.75, 0.72])

array([1.12, 1.1 ])

array([3.37, 3.4 ])

array([4.12, 4.1 ])

array([8.62, 8.6 ])

Define a function and then get correct number for liters

In [78]:
def gallons_to_liters(gallons_list):
    # Conversion factor
    conversion_factor = 3.7854
    # Convert each value in the list from gallons to liters
    liters_list = [round(g * conversion_factor, 2) for g in gallons_list]
    return liters_list

# Given gallons values
gallons_values = [0.01, 0.02, 0.03, 0.06, 0.09, 0.19, 0.29, 0.89, 1.08, 2.27]

# Convert to liters
liters_values = gallons_to_liters(gallons_values)
liters_values

[0.04, 0.08, 0.11, 0.23, 0.34, 0.72, 1.1, 3.37, 4.09, 8.59]

Use map to substitute the wrong values with correct ones

In [79]:
# Given values
gallons_values = [0.01, 0.02, 0.03, 0.06, 0.09, 0.19, 0.29, 0.89, 1.08, 2.27]
liter_values = [0.04, 0.08, 0.11, 0.23, 0.34, 0.72, 1.1, 3.37, 4.09, 8.59]

# Create a dictionary for mapping
gallons_to_liters_map = dict(zip(gallons_values, liter_values))

# Apply the map to the volume_sold_liters column
df6['volume_sold_liters'] = df6['volume_sold_gallons'].map(gallons_to_liters_map).where(
    df6['volume_sold_gallons'].isin(gallons_values), df6['volume_sold_liters']
)

In [12]:
#sanity check
df6['volume_sold_gallons'].nunique()
df6['volume_sold_liters'].nunique()

761

761

Since the two columns now have the same unique counts, we would not do further cleaning so far.

###

### Item Columns Cleaning

In [11]:
crosstab= pd.crosstab(df6['item_number'], df6['item_description'])

In [12]:
columns_with_multiple_nonzero = crosstab.columns[crosstab.gt(0).sum(axis=0) > 1]
columns_with_multiple_nonzero

Index(['1800 Reposado', '1800 Silver', '99 Bananas', '99 Cinnamon',
       '99 Grapes', '99 Mangoes', '99 Oranges', 'Absolut Apeach',
       'Absolut Citron', 'Absolut Mandrin',
       ...
       'Wild Turkey 101 Rye', 'Wild Turkey 81', 'Wild Turkey American Honey',
       'Windsor Canadian', 'Windsor Canadian PET', 'Wisconsin Club Vodka',
       'Wisers Deluxe', 'Woodford Reserve', 'Yukon Jack', 'Zyr Vodka'],
      dtype='object', name='item_description', length=535)

Since we have so many unmatched unique values for item_description, we would like to use item_number as unique column for models.

###

### State Columns Cleaning

In [80]:
crosstab2= pd.crosstab(df6['state_bottle_cost'], df6['state_bottle_retail'])

In [81]:
col_with_multiple_nonzero = crosstab2.columns[crosstab2.gt(0).sum(axis=0) > 1]
col_with_multiple_nonzero

Float64Index([ 7.74,  7.88,   8.4, 10.53, 11.21, 11.52, 16.49,  16.5, 20.63,
              22.05,  30.0,  44.3,  45.0],
             dtype='float64', name='state_bottle_retail')

In [82]:
retail_values = [7.74, 7.88, 8.4, 10.53, 11.21, 11.52, 16.49, 16.5, 20.63, 22.05, 30.0, 44.3, 45.0]

results = {}

# Loop over each 'state_bottle_retail' value and get unique 'state_bottle_cost' values
for value in retail_values:
    unique_state_bottle_cost = df6[df6['state_bottle_retail'] == value]['state_bottle_cost'].unique()
    results[value] = unique_state_bottle_cost

for key, val in results.items():
    print(f"For state_bottle_retail {key}, unique state_bottle_cost values are: {val}")

For state_bottle_retail 7.74, unique state_bottle_cost values are: [ 5.16 60.  ]
For state_bottle_retail 7.88, unique state_bottle_cost values are: [5.25 7.02]
For state_bottle_retail 8.4, unique state_bottle_cost values are: [5.6  7.51]
For state_bottle_retail 10.53, unique state_bottle_cost values are: [ 7.02 25.  ]
For state_bottle_retail 11.21, unique state_bottle_cost values are: [7.47 5.72]
For state_bottle_retail 11.52, unique state_bottle_cost values are: [5.54 7.68]
For state_bottle_retail 16.49, unique state_bottle_cost values are: [10.99 12.49]
For state_bottle_retail 16.5, unique state_bottle_cost values are: [11.    9.66]
For state_bottle_retail 20.63, unique state_bottle_cost values are: [ 13.75  11.     5.94   6.38   8.74  11.07  15.33   9.06 262.46   7.47
   6.  ]
For state_bottle_retail 22.05, unique state_bottle_cost values are: [13.6 14.7]
For state_bottle_retail 30.0, unique state_bottle_cost values are: [20.  17.5]
For state_bottle_retail 44.3, unique state_bottle_

For values like 60 in [ 5.16 60.  ], 60.0 can be a typo of 6.0, thus we would like to choose 5.16 as the unique value of cost.

For values in [ 13.75  11.     5.94   6.38   8.74  11.07  15.33   9.06 262.46   7.47 6.  ], we would like to get the average of those values as unique vlaues.

In [83]:
# Define the list of lists with numbers in each bracket
brackets = [
    [5.25, 7.02],
    [5.6, 7.51],
    [7.47, 5.72],
    [5.54, 7.68],
    [10.99, 12.49],
    [11., 9.66],
    [13.75, 11., 5.94, 6.38, 8.74, 11.07, 15.33, 9.06, 7.47, 6.],
    [13.6, 14.7],
    [20., 17.5],
    [29.53, 11.6],
    [30., 28.2]
]

averages = [sum(bracket) / len(bracket) for bracket in brackets]

for i, average in enumerate(averages):
    print(f"Average for bracket {i + 1}: {average:.3f}")

Average for bracket 1: 6.135
Average for bracket 2: 6.555
Average for bracket 3: 6.595
Average for bracket 4: 6.610
Average for bracket 5: 11.740
Average for bracket 6: 10.330
Average for bracket 7: 9.474
Average for bracket 8: 14.150
Average for bracket 9: 18.750
Average for bracket 10: 20.565
Average for bracket 11: 29.100


In [84]:
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([5.25, 7.02], 6.135)

In [85]:
df6['state_bottle_cost']=df6['state_bottle_cost'].replace(60.  ,5.16)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace(25.  ,7.02)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace( [5.6, 7.51], 6.555)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([7.47, 5.72], 6.595)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([5.54, 7.68], 6.610)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([10.99, 12.49], 11.740)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([11., 9.66], 10.330)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([ 13.75 , 11.   , 5.94 , 6.38 , 8.74 , 11.07 ,15.33  ,9.06,262.46  ,7.47, 6.  ], 9.474)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([13.6, 14.7], 14.150)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([20., 17.5], 18.750)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([29.53, 11.6], 20.565)
df6['state_bottle_cost']=df6['state_bottle_cost'].replace([30., 28.2], 29.100)

In [86]:
df6['state_bottle_cost'].nunique()
df6['state_bottle_retail'].nunique()

1291

1315

In [47]:
# Export df6 to Parquet file
df6.to_parquet('IowaAfterClean3.parquet', index=False)

Next notebook is EDA

In [ ]:
END